In [1]:
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import pandas as pd

In [2]:
def run_double_well(mass, Eo, a, b,
                   temperature, friction, integration_timestep, initial_position,
                   saving_time, simulation_time):
    
    system = openmm.System()
    system.addParticle(mass)
    force = openmm.CustomExternalForce('A*x^4+B*x^2+C*x + D*(y^2+z^2)')
    
    k = 8.0*Eo/(a**2) # stiffness of the armonic potential for coordinates Y and Z
    A = Eo/(a**4)
    B = -2.0*Eo/(a**2)
    C = -b/a
    D = k/2.0
    
    force.addGlobalParameter('A', A)
    force.addGlobalParameter('B', B)
    force.addGlobalParameter('C', C)
    force.addGlobalParameter('D', D)
    
    force.addParticle(0)
    system.addForce(force)
    
    integrator = openmm.LangevinIntegrator(temperature, friction, integration_timestep)
    platform = openmm.Platform.getPlatformByName('CUDA')
    context = openmm.Context(system, integrator, platform)
    
    initial_positions =  np.zeros([1, 3], np.float32) * unit.nanometers
    initial_positions[0,0]= initial_position    
    context.setPositions(initial_positions)

    initial_velocities = np.zeros([1, 3], np.float32) * unit.nanometers/unit.picoseconds
    context.setVelocities(initial_velocities)
    
    saving_steps = int(saving_time/integration_timestep)
    simulation_steps = int(simulation_time/integration_timestep)
    n_saving_periods = int(simulation_steps/saving_steps)
    
    n_frames = n_saving_periods + 1
    trajectory =  np.zeros([n_frames, 1, 3], np.float32) * unit.nanometers
    time = np.zeros([n_frames], np.float32) * unit.nanoseconds
    
    trajectory[0, :, :] = initial_positions
    time[0]= 0 * unit.picoseconds
    
    for ii in range(n_saving_periods):
        integrator.step(saving_steps)
        state = context.getState(getPositions=True)
        trajectory[ii+1, :, :] = state.getPositions()
        time[ii+1]=state.getTime()
    
    return time, trajectory

In [3]:
def get_probability_left (trajectory, barrier=0.0*unit.nanometers):
    occupancy = (trajectory[:, 0, 0] < barrier).sum()
    n_frames = trajectory.shape[0]
    return occupancy/n_frames

def get_probability_right (trajectory, barrier=0.0*unit.nanometers):
    occupancy = (trajectory[:, 0, 0] >= barrier).sum()
    n_frames = trajectory.shape[0]
    return occupancy/n_frames

def average_position (trajectory):
    return trajectory[:, 0, 0].mean()

def standard_deviation_position(trajectory):
    return trajectory[:, 0, 0].std()

In [4]:
mass = 100.0 * unit.amu
Eo=4.0 * unit.kilocalories_per_mole
a=1.0 * unit.nanometers
b=0.0 * unit.kilocalories_per_mole

temperature = 300.0*unit.kelvin
friction = 0.5/unit.picoseconds
integration_timestep = 0.02 * unit.picoseconds

saving_time = 0.1 * unit.picoseconds
simulation_time = 0.5 * unit.nanoseconds

initial_position = 1.0 * unit.nanometers

In [5]:
time, trajectory = run_double_well(mass, Eo, a, b,
                   temperature, friction, integration_timestep, initial_position,
                   saving_time, simulation_time)

In [6]:
get_probability_left(trajectory)

0.0

In [7]:
get_probability_right(trajectory)

1.0

In [8]:
average_position(trajectory)

Quantity(value=0.96286213, unit=nanometer)

In [9]:
standard_deviation_position(trajectory)

Quantity(value=0.1508024, unit=nanometer)

## Lets make some statistics

In [10]:
simulation_times = np.linspace(1.0, 2000.0, num=10) * unit.picoseconds
n_runs = 10

In [11]:
data = [pd.DataFrame(columns=['P_left', 'P_right', 'X_mean', 'X_std']) for ii in simulation_times]

In [12]:
for index in range(len(simulation_times)):
    simulation_time = simulation_times[index]
    print('simulation_time:', simulation_time)
    for run_index in range(n_runs):
        print('     run:',run_index)
        time, trajectory = run_double_well(mass, Eo, a, b,
                   temperature, friction, integration_timestep, initial_position,
                   saving_time, simulation_time)
        data[index].at['run_'+str(run_index),'P_left']=get_probability_left(trajectory)
        data[index].at['run_'+str(run_index),'P_right']=get_probability_right(trajectory)
        data[index].at['run_'+str(run_index),'X_mean']=average_position(trajectory)
        data[index].at['run_'+str(run_index),'X_std']=standard_deviation_position(trajectory)

simulation_time: 1.0 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 223.11111111111111 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 445.22222222222223 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 667.3333333333334 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 889.4444444444445 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 1111.5555555555557 ps
     run: 0
     run: 1
     run: 2
     run: 3
     run: 4
     run: 5
     run: 6
     run: 7
     run: 8
     run: 9
simulation_time: 1333.6666666666667 ps
     run: 0
     run: 1


In [23]:
data[9]

,P_left,P_right,X_mean,X_std
run_0,0,1,0.9614373 nm,0.15872319 nm
run_1,0,1,0.96307784 nm,0.15800579 nm
run_2,0.435578,0.564422,0.12633936 nm,0.96865433 nm
run_3,0,1,0.96693903 nm,0.14951897 nm
run_4,0.481126,0.518874,0.042638846 nm,0.97615075 nm
run_5,0.840658,0.159342,-0.6558025 nm,0.72313696 nm
run_6,0.00129994,0.9987,0.9660811 nm,0.16029705 nm
run_7,0.337883,0.662117,0.31043163 nm,0.9256847 nm
run_8,0.477376,0.522624,0.04185656 nm,0.9747639 nm
run_9,0.558572,0.441428,-0.10706908 nm,0.97329867 nm


In [ ]:
data